In [1]:
%AddJar file:///home/jovyan/data/magpie/dist/Magpie.jar

Starting download from file:///home/jovyan/data/magpie/dist/Magpie.jar
Finished download of Magpie.jar


# Predict New Metallic Glasses
Using the model trained in `build-and-test-gfa-model`, predict new metallic glass alloys. 

In [2]:
import magpie.Magpie;
import magpie.data.Dataset;
import magpie.models.BaseModel;
import magpie.utility.tools.BatchModelEvaluator;
import magpie.data.materials.{CompositionDataset, CompositionEntry};
import magpie.data.utilities.generators.PhaseDiagramCompositionEntryGenerator;
import magpie.data.utilities.filters.{MultiFilter, CompositionSetDistanceFilter, ClassFilter, ClassProbabilityFilter};
import magpie.data.utilities.modifiers.DiscreteToContinuousModifier;
import magpie.optimization.rankers.ClassProbabilityRanker;
import scala.collection.JavaConversions._;

Define the parallelism for this task

In [3]:
Magpie.NThreads = Runtime.getRuntime().availableProcessors();

## Load in the Model
First, load in the model (already trained) from disk

In [4]:
val model = BaseModel.loadState("gfa-model.obj")

## Load in the Training Set
Used for several tasks: defining the search space, generating features, and measuring how 'new' predictions are

In [5]:
val trainData = Dataset.loadState("gfa-dataset.obj").asInstanceOf[CompositionDataset]

## Prepare the Search Space
We want to search all ternary phase diagrams composed of any of the elements the training set. We will evaluate them at 5at% spacing

In [6]:
val gen = new PhaseDiagramCompositionEntryGenerator();

In [7]:
gen.setOrder(1, 3) // 1 - 3 possible components

In [8]:
gen.setSize(51); // 51 elements per binary

In [9]:
gen.setEvenSpacing(true)

Get list of elements

In [10]:
val elems = trainData.getEntries().flatMap(_.asInstanceOf[CompositionEntry].getElements()).map(i => i : java.lang.Integer).to[Set]
println(s"Training set contained ${elems.size} elements")

Training set contained 51 elements


In [11]:
gen.setElementsByIndex(elems)

## Prepare the Filters
Our search space is large enough that storing it all in memory can be a problem. Here, we create filters to determine which entries we should keep

In [12]:
val filter = new MultiFilter()
filter.setExclude(false)

### Filter 1: Only Get Entries with High Confidence
Filter out entries where the confidence in forming a class is less than 0.95

In [13]:
val probFilter = new ClassProbabilityFilter();
probFilter.setThreshold(0.95)
probFilter.setClassName("AM")
probFilter.setExclude(false)

In [14]:
filter.addFilter(probFilter)

### Filter 2: Remove Entries Similar to the Training Set
Remove entries that are less than 30 at% (measured using the $L_1$ distance) from any glass-forming entry in the training set

In [15]:
val glassFilter = new ClassFilter()
glassFilter.setUseMeasured(true)
glassFilter.setThreshold(0)
glassFilter.setComparisonOperator("=")
glassFilter.setExclude(false)
glassFilter.filter(trainData);
println(s"Training data contains ${trainData.NEntries} glass-forming entries")

Training data contains 3801 glass-forming entries


In [20]:
val distFilter = new CompositionSetDistanceFilter()
distFilter.setDistanceThreshold(0.3)
distFilter.setCompositions(trainData)
distFilter.setP(1)
distFilter.setExclude(false)

In [17]:
filter.addFilter(distFilter)

## Create the Batch Generator
We will evaluate entries in batches of $10^5$ to save on memory consumption. What this class does is automate the process of generating batches of entries in the search space, computing their features, using the ML model to classify them, and filtering out the desired predictions. 

In [18]:
val runner = new BatchModelEvaluator()

In [19]:
runner.setBatchSize(10000)

In [20]:
runner.setModel(model)

## Run the Search!
Evaluate all of the entries

In [21]:
val searchResults = trainData.emptyClone()
runner.evaluate(searchResults, gen, filter)

In [22]:
println(s"Found ${searchResults.NEntries} likely glasses")

Found 73397 new predictions


Store the probabilities

In [23]:
val mdfr = new DiscreteToContinuousModifier();
mdfr.setClassName("AM")
mdfr.transform(searchResults)

In [24]:
val filename = searchResults.saveCommand("new_glasses", "comp")